In [ ]:
# Clone the GitHub repo so all bot code is automatically available
import subprocess
import sys
import os
from pathlib import Path

REPO_URL = "https://github.com/amer6767/Clawd-bot.git"
CLONE_DIR = "/kaggle/working/Clawd-bot"

if Path(CLONE_DIR).exists():
    print(f"Repo already cloned at {CLONE_DIR}, pulling latest changes...")
    subprocess.run(["git", "-C", CLONE_DIR, "pull"], check=True)
else:
    print(f"Cloning {REPO_URL} into {CLONE_DIR}...")
    subprocess.run(["git", "clone", REPO_URL, CLONE_DIR], check=True)

# Add the repo root to Python path so 'territorial_bot' package is importable
if CLONE_DIR not in sys.path:
    sys.path.insert(0, CLONE_DIR)

print(f"territorial_bot path added: {CLONE_DIR}")
print("Repo setup complete!")

In [ ]:
# Install all dependencies
import subprocess
subprocess.run(["pip", "install", "-q", "playwright", "opencv-python-headless", "Pillow", "numpy", "torch", "torchvision", "scikit-learn", "tqdm", "pyngrok", "flask", "flask-cors"], check=True)
subprocess.run(["playwright", "install", "chromium"], check=True)
subprocess.run(["playwright", "install-deps", "chromium"], check=True)
print("All dependencies installed!")

In [ ]:
import sys
import os
import asyncio
import threading
import time
import base64
import json
from pathlib import Path
from flask import Flask, jsonify, request
from flask_cors import CORS
from pyngrok import ngrok

# Working directory
os.chdir('/kaggle/working')

# Create required directories
Path('/kaggle/working/Clawd-bot/territorial_bot/logs').mkdir(parents=True, exist_ok=True)
Path('/kaggle/working/Clawd-bot/territorial_bot/models').mkdir(parents=True, exist_ok=True)
Path('/kaggle/working/Clawd-bot/territorial_bot/screenshots').mkdir(parents=True, exist_ok=True)

print("Setup complete!")

## Bot is Ready

The GitHub repo has been cloned automatically in cell 1. All bot files are available from:
`/kaggle/working/Clawd-bot/territorial_bot/`

No manual file uploads needed!

In [ ]:
# Global bot state
bot_state = {
    "running": False,
    "status": "idle",
    "wins": 0,
    "losses": 0,
    "territory_pct": 0.0,
    "score": 0,
    "last_update": time.time(),
    "screenshot_b64": "",
    "error": None
}

bot_controller = None
bot_thread = None
bot_loop = None

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

@app.route('/start', methods=['POST', 'GET'])
def start_bot():
    global bot_controller, bot_thread, bot_loop, bot_state
    if bot_state["running"]:
        return jsonify({"status": "already_running", "message": "Bot is already running"})
    
    bot_state["running"] = True
    bot_state["status"] = "starting"
    bot_state["error"] = None
    
    def run_bot():
        global bot_controller, bot_loop, bot_state
        try:
            from territorial_bot.bot_controller import BotController
            bot_loop = asyncio.new_event_loop()
            asyncio.set_event_loop(bot_loop)
            bot_controller = BotController(player_name="KaggleBot", headless=True)
            bot_state["status"] = "running"
            bot_loop.run_until_complete(bot_controller._run())
        except Exception as e:
            bot_state["error"] = str(e)
            bot_state["status"] = "error"
        finally:
            bot_state["running"] = False
            bot_state["status"] = "stopped"
    
    bot_thread = threading.Thread(target=run_bot, daemon=True)
    bot_thread.start()
    return jsonify({"status": "started", "message": "Bot starting up..."})

@app.route('/stop', methods=['POST', 'GET'])
def stop_bot():
    global bot_controller, bot_state
    if not bot_state["running"]:
        return jsonify({"status": "not_running", "message": "Bot is not running"})
    
    try:
        if bot_controller:
            bot_controller.stop()
        bot_state["running"] = False
        bot_state["status"] = "stopped"
        return jsonify({"status": "stopped", "message": "Bot stopped"})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)})

@app.route('/status', methods=['GET'])
def get_status():
    return jsonify({
        "running": bot_state["running"],
        "status": bot_state["status"],
        "last_update": bot_state["last_update"],
        "error": bot_state["error"]
    })

@app.route('/screenshot', methods=['GET'])
def get_screenshot():
    screenshot_dir = Path('/kaggle/working/Clawd-bot/territorial_bot/screenshots')
    screenshots = sorted(screenshot_dir.glob('*.png'), key=lambda p: p.stat().st_mtime, reverse=True)
    if screenshots:
        with open(screenshots[0], 'rb') as f:
            img_b64 = base64.b64encode(f.read()).decode('utf-8')
        return jsonify({"screenshot": img_b64, "timestamp": time.time()})
    return jsonify({"screenshot": "", "timestamp": time.time(), "message": "No screenshot available"})

@app.route('/stats', methods=['GET'])
def get_stats():
    stats_path = Path('/kaggle/working/Clawd-bot/territorial_bot/models/learning_data.json')
    stats = {
        "wins": bot_state["wins"],
        "losses": bot_state["losses"],
        "territory_pct": bot_state["territory_pct"],
        "score": bot_state["score"],
        "status": bot_state["status"]
    }
    if stats_path.exists():
        try:
            with open(stats_path) as f:
                data = json.load(f)
            stats["total_games"] = data.get("total_games", 0)
            stats["wins"] = data.get("total_wins", stats["wins"])
            stats["losses"] = data.get("total_losses", stats["losses"])
        except Exception:
            pass
    return jsonify(stats)

@app.route('/', methods=['GET'])
def index():
    return jsonify({"message": "Territorial Bot API", "endpoints": ["/start", "/stop", "/status", "/screenshot", "/stats"]})

print("Flask app defined!")

In [ ]:
# Read ngrok auth token from Kaggle secret / environment variable
NGROK_AUTH_TOKEN = os.environ.get("NGROK_TOKEN", "")
if not NGROK_AUTH_TOKEN:
    raise ValueError(
        "NGROK_TOKEN environment variable is not set. "
        "Go to Kaggle -> Add-ons -> Secrets and add NGROK_TOKEN with your token from https://ngrok.com"
    )

# Configure ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start Flask in background thread
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()
time.sleep(2)

# Create ngrok tunnel
public_url = ngrok.connect(5000)
print(f"\n{'='*60}")
print(f"\U0001f680 Bot API is LIVE!")
print(f"\U0001f4f1 Your ngrok URL: {public_url}")
print(f"{'='*60}")
print(f"\nAPI Endpoints:")
print(f"  Start bot:   {public_url}/start")
print(f"  Stop bot:    {public_url}/stop")
print(f"  Status:      {public_url}/status")
print(f"  Screenshot:  {public_url}/screenshot")
print(f"  Stats:       {public_url}/stats")
print(f"\n\u26a0\ufe0f  Copy the ngrok URL above and paste it into your mobile website!")

In [ ]:
# Keep the notebook running (run this cell to keep the API alive)
print("API is running. Press the stop button to shut down.")
print("Check status at: /status")
while True:
    time.sleep(60)
    print(f"[{time.strftime('%H:%M:%S')}] API still running... Bot status: {bot_state['status']}")